In [ ]:
#change dir to SynthText-English
import os
os.chdir('SynthText-English/')

# current_dir = os.getcwd()
# print("Current directory:", current_dir)

Current directory: /Users/myatthirimaung/Documents/Studies/UniversityStudies/Semester1_2025-2026/IT_project/Convert-SynthText-to-PaddleOCR-format-Train-SVTR/SynthText-English


In [12]:
from __future__ import division
import os
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt 
import h5py 
# from common import *

from natsort import natsorted
import itertools
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import regex as re
import tqdm

In [13]:
def prepare_text(text):
    '''general text preparation'''
    text = text.replace('\n', ' ').replace('\t', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [15]:
#load generated .h5 file
db = h5py.File('SynthText-EN.h5', 'r')

dsets = natsorted(db['data'].keys())
print ("total number of images : ", len(dsets))

total number of images :  5000


In [16]:
#make dirs
os.makedirs(f'scene_text_recognition/train_images/', exist_ok=True)
os.makedirs(f'scene_text_recognition/val_images/', exist_ok=True)

In [17]:
#val images
len_val_images = int(len(dsets) * 0.1)
len_val_images

500

In [18]:
#train images
len_train_images = len(dsets) - len_val_images
len_train_images

4500

In [19]:
#train images crop generation

im_n = 0
im_name_label = []

for item in tqdm.tqdm_notebook(list(range(len(dsets)))[0:len_train_images]):
    
    k = dsets[item]
    
    rgb = db['data'][k][...]
    wordBB = db['data'][k].attrs['wordBB']
    txt = db['data'][k].attrs['txt']
    txt = [prepare_text(i.decode('utf-8')).split(' ') for i in txt]
    txt = list(itertools.chain(*txt))
    
    #print(txt)
    
    # plot the word-BB:
    for image_item, im_text in zip(range(wordBB.shape[-1]), txt):
        
        bb = wordBB[:,:,image_item]
        bb = np.c_[bb,bb[:,0]]

        img_cutted = rgb[int(min(bb[1])):int(max(bb[1])), int(min(bb[0])):int(max(bb[0]))]

        im_name = f'{im_n}.png'
        
        try:
            cv2.imwrite(f'scene_text_recognition/train_images/{im_name}', img_cutted, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            im_name_label.append([im_name, im_text])
        except:
            print(OSError)
            im_n += 1
            continue
        #except:
        #    print('Error!')

        im_n += 1

/var/folders/d3/msq7vg0s0zn5dk94qw85y4dh0000gn/T/ipykernel_16984/1755644686.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for item in tqdm.tqdm_notebook(list(range(len(dsets)))[0:len_train_images]):


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [81]:
#train df
columns = ['file_name','text']       
train_df = pd.DataFrame(im_name_label, columns=columns)
train_df

,file_name,text
0,1.png,2669
1,2.png,black
2,3.png,MARHESHVAN
3,4.png,Сладенько
4,6.png,GONAD
...,...,...
3916,3999.png,Ratherish
3917,4000.png,скрипну]
3918,4001.png,Cubitometacarpal
3919,4002.png,непросвещённое


In [82]:
#val images crop generation

im_n = 0
im_name_label = []

for item in tqdm.tqdm_notebook(list(range(len(dsets)))[len_train_images:]):
    
    k = dsets[item]
    
    rgb = db['data'][k][...]
    wordBB = db['data'][k].attrs['wordBB']
    txt = db['data'][k].attrs['txt']
    txt = [prepare_text(i.decode('utf-8')).split(' ') for i in txt]
    txt = list(itertools.chain(*txt))
    
    #print(txt)
    
    # plot the word-BB:
    for image_item, im_text in zip(range(wordBB.shape[-1]), txt):
        
        bb = wordBB[:,:,image_item]
        bb = np.c_[bb,bb[:,0]]

        img_cutted = rgb[int(min(bb[1])):int(max(bb[1])), int(min(bb[0])):int(max(bb[0]))]

        im_name = f'{im_n}.png'
        
        try:
            cv2.imwrite(f'scene_text_recognition/val_images/{im_name}', img_cutted, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            im_name_label.append([im_name, im_text])
        except:
            print(OSError)
            im_n += 1
            continue
        #except:
        #    print('Error!')

        im_n += 1

<ipython-input-82-390b68b16fa8>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for item in tqdm.tqdm_notebook(list(range(len(dsets)))[len_train_images:]):


  0%|          | 0/88 [00:00<?, ?it/s]

<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>
<class 'OSError'>


In [83]:
#val df
columns = ['file_name','text']       
val_df = pd.DataFrame(im_name_label, columns=columns)
val_df

,file_name,text
0,0.png,False
1,1.png,1067
2,2.png,Tom
3,3.png,2508
4,4.png,2742
...,...,...
430,442.png,348
431,443.png,312
432,444.png,Schoolery
433,445.png,pre


In [84]:
#train and val .txt files

with open("scene_text_recognition/train.txt", "w", encoding='utf8') as fo:        
    for index in tqdm.tqdm_notebook(range(train_df.shape[0])):
        file_name = train_df.iloc[index][0]
        text = train_df.iloc[index][1]
        fo.write(file_name + '\t' + text + '\n')
        
with open("scene_text_recognition/val.txt", "w", encoding='utf8') as fo:        
    for index in tqdm.tqdm_notebook(range(val_df.shape[0])):
        file_name = val_df.iloc[index][0]
        text = val_df.iloc[index][1]
        fo.write(file_name + '\t' + text + '\n')

<ipython-input-84-e7276f72c85e>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm.tqdm_notebook(range(train_df.shape[0])):


  0%|          | 0/3921 [00:00<?, ?it/s]

<ipython-input-84-e7276f72c85e>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm.tqdm_notebook(range(val_df.shape[0])):


  0%|          | 0/435 [00:00<?, ?it/s]

In [85]:
#make archive for training text recognition using PaddleOCR
import shutil
shutil.make_archive('scene_text_recognition', 'zip', 'scene_text_recognition')

'/Users/datanomica/Downloads/SynthText-master/scene_text_recognition.zip'